# einsum dev

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from numpy.core.einsumfunc import _parse_einsum_input
from mygrad.linalg.einsum import einsum

In [4]:
from decimal import Decimal
from decimal import getcontext
getcontext().prec = 25

def to_decimal_array(arr):
    """ Convert numpy ND-array to Decimal-type object array of the same shape.
        Used for facilitating high-precision arithmetic.

        Parameters
        ----------
        arr : Union[float, numpy.ndarray]

        Returns
        -------
        numpy.ndarray
        Decimal-type object array"""
    arr = np.asarray(arr)
    return np.array(tuple(Decimal(float(i)) for i in arr.flat), dtype=Decimal).reshape(arr.shape)


def numerical_gradient_sequence(f, *, x, back_grad,  h=1e-8):
    """ Computes numerical partial derivatives of f({x}), where f is a numpy-style
        sequential function (e.g. numpy.sum, numpy.mean, ...). The partial derivative
        is computed for each member of {x}

        Parameters
        ----------
        f : Callable[[numpy.ndarray], numpy.ndarray]
            f({x}) -> numpy.ndarray

        x : numpy.ndarray
            An array storing the sequence(s) of values in the array. More than once
            sequence may be designated, according to the `axis` argument of `f`.

        axis : Optional[None, int, Tuple[int, ...]]
            The value of the `axis` argument, to be fed to `f`.

        keepdims : bool, optional (default=False)
            The value of the `keepdims` argument, to be fed to `f`.

        back_grad : numpy.ndarray
            The gradient being back-propagated to {x}, via f

        h : float, optional, (default=1e-8)
            Approximating infinitesimal.

        Returns
        -------
        numpy.ndarray
            df/d{x}
        """

    grad = np.empty_like(x)
    x = to_decimal_array(x)
    h = Decimal(h)

    for ind, val in np.ndenumerate(x):
        x_fwd = np.copy(x)
        x_fwd[ind] += h
        f_fwd = to_decimal_array(f(x_fwd.astype(float)))

        x_bkwd = x_fwd
        x_bkwd[ind] -= Decimal(2) * h
        f_bkwd = to_decimal_array(f(x_bkwd.astype(float)))

        dxi = to_decimal_array((f_fwd - f_bkwd) / (Decimal(2) * h))
        grad[ind] = (dxi.astype('float') * back_grad).sum()
    return grad.astype(float)

def backprop_linalg(f, *args, back_grad):
    grads = []
    args = tuple(i for i in args)
    for n in range(len(args)):
        tmp_f = lambda var: f(*args[:n], var, *args[n+1:])
        grads.append(numerical_gradient_sequence(tmp_f, x=args[n], back_grad=back_grad))
    return grads

\begin{equation}
f = X_i Y_i\\
\frac{\partial L}{\partial X_a} = \frac{\partial L}{\partial f}Y_a\\
\frac{\partial L}{\partial Y_a} = X_a\frac{\partial L}{\partial f}\\
\end{equation}

In [18]:
x = np.random.rand(3)
y = np.random.rand(3)
grad = np.random.rand(1).item()


# fwd pass function
np.allclose(np.einsum("i, i", x, y), np.dot(x, y))
def f(x, y): return np.einsum("i, i", x, y)
dfdx = np.einsum("..., i", grad, y)
dfdy = np.einsum("i, ...", x, grad)
dx, dy = backprop_linalg(f , x, y, back_grad=grad)

assert np.allclose(dfdx, dx)
assert np.allclose(dfdy, dy) 

In [13]:
np.einsum(a)

ValueError: must provide at least an operand and a subscripts list to einsum

In [14]:
>>> a = np.arange(25).reshape(5,5)
>>> b = np.arange(5)
>>> c = np.arange(6).reshape(2,3)

_parse_einsum_input((a, [0,0], [0]))

('aa', 'a', [array([[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24]])])

In [6]:
einsum("i, i", x, y)

Tensor(0.3458595614583897)

\begin{equation}
f_a = X_{ia} Y_i\\
\frac{\partial L}{\partial X_{ia}} = \big[\frac{\partial L}{\partial f}\big]_aY_i\\
\frac{\partial L}{\partial Y_i} = X_{ia}\big[\frac{\partial L}{\partial f}\big]_{a}\\
\end{equation}

In [20]:
_parse_einsum_input((a, [0, 0]))

('aa', '', [array([[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24]])])

In [30]:
>>> a = np.arange(6).reshape((3,2))
>>> b = np.arange(12).reshape((4,3))
>>> np.einsum('k...,jk', a, b)

array([[10, 28, 46, 64],
       [13, 40, 67, 94]])

In [31]:
np.einsum(a, [0, Ellipsis], b, [2, 0])

array([[10, 28, 46, 64],
       [13, 40, 67, 94]])

In [16]:
_parse_einsum_input(("ia, i -> a", x, y))

('ia,i', 'a', [array([[0.53369578, 0.52774017, 0.56496211, 0.54836422],
         [0.84556295, 0.94586904, 0.86975989, 0.52495072],
         [0.98848596, 0.42243614, 0.25886365, 0.8120052 ]]),
  array([0.16823345, 0.99113357, 0.68551716])])

In [15]:
x = np.random.rand(3, 4)
y = np.random.rand(3)
grad = np.random.rand(4)


def f(x, y): return np.einsum("ia, i -> a", x, y)

dfdx = np.einsum("a, i -> ia", grad, y)
dfdy = np.einsum("ia, a", x, grad)

dx, dy = backprop_linalg(f , x, y, back_grad=grad)

assert np.allclose(dfdx, dx)
assert np.allclose(dfdy, dy)

\begin{equation}
f_{ab} = X_{ai} Y_{ib}\\
\frac{\partial L}{\partial X_{ai}} = \big[\frac{\partial L}{\partial f}\big]_{ab}Y_{ib}\\
\frac{\partial L}{\partial Y_{ib}} = X_{ai}\big[\frac{\partial L}{\partial f}\big]_{ab}\\
\end{equation}

In [16]:
x = np.random.rand(3, 4)
y = np.random.rand(4, 2)
grad = np.random.rand(3, 2)

# fwd pass function
def f(x, y): return np.einsum("ai, ib", x, y)

dfdx = np.einsum("ab, ib", grad, y)
dfdy = np.einsum("ai, ab -> ib", x, grad)

dx, dy = backprop_linalg(f, x, y, back_grad=grad)

assert np.allclose(dfdx, dx)
assert np.allclose(dfdy, dy)

\begin{equation}
f_{aBCd} = X_{iBCj} Y_{aijd}\\
\frac{\partial L}{\partial X_{iBCj}} = \big[\frac{\partial L}{\partial f}\big]_{aBCd}Y_{aijd}\\
\frac{\partial L}{\partial Y_{aijd}} = X_{iBCj}\big[\frac{\partial L}{\partial f}\big]_{aBCd}\\
\end{equation}

In [19]:
x = np.random.rand(6, 3, 4, 7)
y = np.random.rand(1, 6, 7, 2)
grad = np.random.rand(1, 3, 4, 2)

def f(x, y): return np.einsum("iBCj, aijd -> aBCd", x, y)

dfdx = np.einsum("aBCd, aijd -> iBCj", grad, y)
dfdy = np.einsum("aBCd, iBCj -> aijd", grad, x)

dx, dy = backprop_linalg(f, x, y, back_grad=grad)

assert np.allclose(dfdx, dx)
assert np.allclose(dfdy, dy)

In [82]:
np.einsum("ibcj,aijd->abcd", x, y)

array([[[[ 9.75961412, 11.2738716 ],
         [10.54329569, 11.45274101],
         [13.04174456, 15.32372693],
         [12.50578688, 14.12400693]],

        [[10.51790034, 11.04253852],
         [12.02509079, 12.74567501],
         [10.95296063, 14.16075399],
         [ 9.65389081, 10.95986723]],

        [[10.04625651, 12.30753986],
         [10.97285038, 13.21286666],
         [12.28231682, 12.73070599],
         [10.50828129, 11.80353908]]]])

In [29]:
def arr(*shape): return np.random.rand(*shape)

In [76]:
x = arr(3,2,4)
y = arr(1,1,4)

In [63]:
(x*y)

array([[[0.01193077, 0.352678  , 0.48184998, 0.124214  ],
        [0.00319909, 0.18915548, 0.05267074, 0.10196847]],

       [[0.03967746, 0.33860053, 0.36308553, 0.281475  ],
        [0.00681724, 0.45875322, 0.13282656, 0.2306132 ]],

       [[0.0443181 , 0.54715235, 0.48651136, 0.28359557],
        [0.00366073, 0.15340006, 0.00898187, 0.77429298]]])

In [72]:
x = arr(3)

In [74]:
np.einsum("i->ii", x)

ValueError: einstein sum subscripts string includes output subscript 'i' multiple times

In [77]:
np.einsum("ijk,...->...", x, y)

array([[[ 8.38892778, 11.24371902,  4.12828738, 11.69130208]]])

In [81]:
np.tensordot(x, y, [[0, 3], [1, 2]]).transpose(2, 0, 1, 3)

array([[[[ 9.75961412, 11.2738716 ],
         [10.54329569, 11.45274101],
         [13.04174456, 15.32372693],
         [12.50578688, 14.12400693]],

        [[10.51790034, 11.04253852],
         [12.02509079, 12.74567501],
         [10.95296063, 14.16075399],
         [ 9.65389081, 10.95986723]],

        [[10.04625651, 12.30753986],
         [10.97285038, 13.21286666],
         [12.28231682, 12.73070599],
         [10.50828129, 11.80353908]]]])

In [42]:
dx, dy = backprop_linalg(np.dot, x, y, back_grad=grad)

In [31]:
numerical_gradient_sequence(fx, x=x, back_grad=grad)

array([[1.23289217, 0.30646453, 0.51300205, 0.88965838],
       [0.60798617, 0.24371856, 0.07130679, 0.73327706],
       [0.47803916, 0.17882325, 0.08119049, 0.53581647]])

In [43]:
np.allclose(np.einsum("ai,bi", grad, y), dx)

True

In [34]:
numerical_gradient_sequence(fy, x=y, back_grad=grad)

array([[0.60317337, 0.28455023],
       [0.46700131, 0.28474071],
       [1.22450942, 0.90343718],
       [0.90322037, 0.21694252]])

In [44]:
np.allclose(np.einsum("ia,ib", x, grad), dy)

True